# Web Auth Signing を使用した AgentCore Browser

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Browser ツールで Web Bot Auth 署名を有効にする方法を学びます。

### チュートリアル詳細

| 項目         | 詳細                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ       | 会話型                                                                   |
| エージェントタイプ          | シングル                                                                           |
| エージェントフレームワーク   | Strands                                                                          |
| LLM モデル           | Claude 4.5 Haiku                                                                 |
| チュートリアルコンポーネント | ブラウザ自動化、ブラウザリクエストの *Web Bot Auth* 署名                   |
| チュートリアル分野   |                                                                                  |
| 難易度  | 中級                                                                     |
| 使用 SDK            | Amazon Bedrock AgentCore Python SDK、Strands Agents、Strands Agent Tools         |
 
### チュートリアルアーキテクチャ

![Architecture](images/Architecture.png)

### チュートリアルの主な機能

* Strands エージェントでのヘッドレスブラウザツールの使用
* strands_tools AgentCoreBrowser ツールの使用
* 高速で効率的な分析のための Claude 4.5 Haiku モデル

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報の設定
* Amazon Bedrock AgentCore SDK
* Strands agents および strands-agents-tools パッケージ
* Amazon Bedrock の Claude 4.5 Haiku モデルへのアクセス

**実装**: ブラウザ自動化のための公式 `strands_tools.browser.AgentCoreBrowser` をエージェントから使用します。

## AgentCore Browser の設定

AgentCore Browser ツールを使用する前に、記録機能、ネットワーク設定、実行ロールなどの特定の設定でカスタムブラウザ設定を作成できます。このセクションでは、AWS SDK を使用してカスタムブラウザ設定を作成する方法を示します。

### カスタムブラウザ設定の作成

以下のコードは、記録を有効にし、特定の実行ロールを持つカスタム AgentCore Browser を作成する方法を示しています。

#### Web Bot Auth 署名を使用した Browser 設定の作成

**Browser 署名とは？**

Browser 署名（`browserSigning.enabled = True`）は、AgentCore Browser がすべての送信 HTTP リクエストを自動的に署名するように設定します。これは以下の目的で不可欠です：

- **認証済み API アクセス**: 保護された API へのリクエストに署名
- **AWS サービス統合**: AWS 認証情報でリクエストを自動署名
- **セキュリティコンプライアンス**: すべてのブラウザリクエストが認証されることを保証
- **リクエストの整合性**: リクエストヘッダーの暗号署名

有効にすると、ブラウザは以下を行います：
1. すべての HTTP/HTTPS リクエストをインターセプト
2. リクエストヘッダーに暗号署名を追加
3. 認証トークンを自動的に含める
4. リクエスト全体でセッションの整合性を維持

### ライブラリのインポート

In [ ]:
import boto3
import uuid
import os
import sys
from strands import Agent
from strands_tools.browser import AgentCoreBrowser
import asyncio
import time

tutorials_path = os.path.abspath(os.path.join(os.getcwd(), '../../../'))
if tutorials_path not in sys.path:
    sys.path.insert(0, tutorials_path)

from utils import create_agentcore_role

cp_client = boto3.client('bedrock-agentcore-control', 
                         region_name='us-west-2')


accountId = boto3.client("sts").get_caller_identity()["Account"] 
region = boto3.Session().region_name

print(f"アカウント ID: {accountId}")
print(f"リージョン: {region}")

### セットアップとロールの作成

In [ ]:
## 実行ロールを作成
execution_role_arn = create_agentcore_role("web-bot-auth")["Role"]["Arn"]

print(f"\n✅ ロール作成成功: {execution_role_arn}")

## カスタム設定で新しいブラウザインスタンスを作成
response = cp_client.create_browser(
    name="web_bot_auth_browser_" + str(uuid.uuid4())[:6],
    description="Browser configured to sign web bot auth",
    networkConfiguration={
        "networkMode": "PUBLIC"
    },
    executionRoleArn=execution_role_arn,
    browserSigning={
        "enabled": True
    }
)

browserId = response['browserId']
browserArn = response['browserArn']
print(f"\n✅ Browser 作成成功！")
print(f"   Browser ID: {browserId}")
print(f"   Browser ARN: {browserArn}")
print(f"\n🔐 Browser 署名が有効 - すべてのリクエストが自動的に署名されます")

### AgentCoreBrowser ツールを使用した Strands Agent の作成

In [ ]:
# AgentCoreBrowser を使用して Strands エージェントを作成・設定
# 前のステップで作成したカスタム Browser ID を使用して公式 AgentCoreBrowser ツールを初期化
agent_core_browser = AgentCoreBrowser(identifier=browserId, region="us-west-2")
agent_core_default_browser = AgentCoreBrowser(region="us-west-2")

# 並行ブラウザ操作を防ぐために SequentialToolExecutor をインポート
from strands.tools.executors import SequentialToolExecutor
    
# Claude 4.5 Haiku モデルと SEQUENTIAL ツール実行で署名付きエージェントを作成
strands_agent = Agent(
    tools=[agent_core_browser.browser],  # 署名が有効なカスタムブラウザを使用
    tool_executor=SequentialToolExecutor(),  # 並行ブラウザ操作を防止
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    system_prompt="""あなたはブラウザ署名機能を持つウェブサイトアナリストです。
1. ブラウザツールを使用してウェブサイトを効率的に訪問し操作してください
2. 2-3回のブラウザ操作で素早く重要な情報を抽出することに集中してください
3. Web Bot Auth の Signature および Signature-Agent HTTP ヘッダーに関連する署名についてブラウザリクエストを確認し、ブラウザ署名が設定されているかを検証してください"""
)

# Claude 4.5 Haiku モデルと SEQUENTIAL ツール実行で署名なしエージェントを作成
strands_agent_unsigned = Agent(
    tools=[agent_core_default_browser.browser],  # 署名なしのデフォルトブラウザを使用
    tool_executor=SequentialToolExecutor(),  # 並行ブラウザ操作を防止
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    system_prompt="""あなたはブラウザ署名機能を持つウェブサイトアナリストです。
1. ブラウザツールを使用してウェブサイトを効率的に訪問し操作してください
2. 2-3回のブラウザ操作で素早く重要な情報を抽出することに集中してください
3. Web Bot Auth の Signature および Signature-Agent HTTP ヘッダーに関連する署名についてブラウザリクエストを確認し、ブラウザ署名が設定されているかを検証してください"""
)

In [ ]:
# 順次実行で非同期関数を定義（並行競合なし）
async def analyze_website(agent, prompt):
    """順次ツール実行でのエージェント呼び出しの非同期ラッパー"""
    try:
        # SequentialToolExecutor により、ブラウザ操作は競合しない
        result = await agent.invoke_async(prompt)
        return result
    except Exception as e:
        print(f"❌ エラー: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

### Browser 署名を有効にした分析の開始

In [ ]:
print("\n🚀 CloudFlare の crawltest サイトで browser 署名を検証します。")
print("="*100)
  
result_signed = await analyze_website(
    strands_agent,
    "Review the output and status code at https://crawltest.com/cdn-cgi/web-bot-auth and provide 3 to 4 concise key insights, based on https://developers.cloudflare.com/bots/reference/bot-verification/web-bot-auth/"
)

if result_signed:
    print("\n\n✅ Web Bot Auth browser 署名有効で分析完了")
    print("-"*100)
    print(result_signed)
    print("-"*100)

### Browser 署名なしで実験を再実行

テストが有効であることを検証するため、同じプロンプトを browser 署名なしで設定されたエージェントで実行してみましょう。

In [ ]:
print("\n🚀 CloudFlare の crawltest サイトで browser 署名を検証 - Web Bot Auth browser 署名なし。")
print("="*100)
  
result_unsigned = await analyze_website(
    strands_agent_unsigned,
    "Review the output and status code at https://crawltest.com/cdn-cgi/web-bot-auth and provide 3 to 4 concise key insights, based on https://developers.cloudflare.com/bots/reference/bot-verification/web-bot-auth/"
)

if result_unsigned:
    print("\n\n✅ Web Bot Auth browser 署名なしで分析完了")
    print("-"*100)
    print(result_unsigned)
    print("-"*100)

結果を比較しましょう：

In [ ]:
# Strands Agent を使用して result_unsigned と result_signed を比較
comparison_prompt = f"""
以下の2つのエージェント出力を並べて分析・比較してください：

**署名なしエージェントの出力:**
{result_unsigned}

**署名付きエージェントの出力:**
{result_signed}

以下を提供してください：
* 主要な違いを強調した並列比較
* 署名付きエージェントが署名を正しく使用していることの検証
* 署名なしエージェントが署名を使用していないことの検証
* 最も大きく異なる側面の要約

読みやすいように見出しと箇条書きで明確に回答をフォーマットしてください。
"""

# Strands Agent を呼び出して結果を比較
comparison_agent = Agent(
    system_prompt="""あなたは異なる AI エージェントの出力を評価する専門データアナリストです。""",
    callback_handler=None
)

comparison_response = comparison_agent(comparison_prompt)

print("=== エージェント出力の比較 ===")
print(comparison_response)

## 裏側で何が起きていたか

**browser 署名を有効にして**このノートブックを実行すると、以下のプロセスが発生します：

### 1. **Browser 設定の作成**
```python
browserSigning={
    "enabled": True
}
```
これは、AgentCore Browser サービスにブラウザによるすべての HTTP リクエストを自動的に署名するよう指示します。

### 2. **エージェントの初期化**
Strands エージェントは以下で初期化されます：
- 署名が有効な browser 設定識別子。*注意* これはデフォルトの browser 識別子ではありません。
- Claude 4.5 Haiku モデル
- Browser ツール統合

### 3. **リクエスト署名フロー**
エージェントがウェブサイトにナビゲートすると：

```text
┌─────────────────┐    ┌───────────────────┐    ┌──────────────────────┐    ┌──────────────┐
│  Agent Request  │ ──→│ AgentCore Browser │ ──→│ Sign Request Headers │ ──→│Target Website│
└─────────────────┘    └───────────────────┘    └──────────────────────┘    └──────────────┘
                                                          │
                                                          ▼
                                                Web Bot Auth ヘッダーを追加:
                                                • Signature-Input ヘッダー
                                                • Signature-Agent ヘッダー
                                                • Signature ヘッダー

```

## セキュリティ上の利点

browser 署名を有効にすると：

- **自動認証** - 手動のヘッダー管理が不要  
- **リクエストの整合性** - 暗号署名により改ざんを防止  
- **AWS 統合** - エージェントが browser ツールを使用するためのシームレスな IAM 認証情報統合 
- **セッション管理** - 各セッションが独自のブラウザセッションを持つ安全なセッショントークン処理 
- **監査証跡** - すべての署名付きリクエストをログに記録可能  

## トラブルシューティング

### エラー: "Browser session not found"
**解決策**: browser ID が期限切れになっている可能性があります。browser 作成セル（cell-3）を再実行してください

### RuntimeError: "Leaving task does not match the current task"
**解決策**: この asyncio エラーはブラウザ操作が並行して実行されるときに発生します。このノートブックでは `SequentialToolExecutor()` を使用してブラウザ操作を1つずつ順番に実行することでこの問題を防いでいます。

**なぜこれが起きるか**: エージェントが AgentCore Browser ツールに複数の非同期操作（get_html、get_text、screenshot など）を並行して実行させると、Strands がそれらを同時に実行し、競合する asyncio タスクを作成する可能性があります。

**修正方法**: `SequentialToolExecutor()` を使用してすべてのツール呼び出しを順番に実行し、同じ機能を維持しながら非同期タスクの競合を排除します。

## 参考資料

Web Bot Auth と AI エージェントの CAPTCHA を削減する方法についてさらに学ぶには、以下のリソースをご確認ください：

### AWS ブログ記事
- **[Reduce CAPTCHAs for AI agents browsing the web with Web Bot Auth (Preview) in Amazon Bedrock AgentCore Browser](https://aws.amazon.com/blogs/machine-learning/reduce-captchas-for-ai-agents-browsing-the-web-with-web-bot-auth-preview-in-amazon-bedrock-agentcore-browser/)** - Web Bot Auth の実装と利点に関する包括的なガイド

### ドキュメント
- **[Cloudflare Web Bot Auth Documentation](https://developers.cloudflare.com/bots/reference/bot-verification/web-bot-auth/)** - 技術仕様と実装の詳細
- **[Amazon Bedrock AgentCore Browser Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-onboarding.html)** - AgentCore Browser 機能の完全ガイド

### 関連トピック
- **[HTTP Message Signatures (RFC 9421)](https://datatracker.ietf.org/doc/rfc9421/)** - Web Bot Auth で使用される基礎となる暗号標準
- **[Web Bot Auth Architecture (IETF Draft)](https://datatracker.ietf.org/doc/html/draft-meunier-web-bot-auth-architecture)** - Web Bot Auth アーキテクチャの IETF ドラフト仕様